In [28]:
import pandas as pd
import matplotlib.pyplot as plt
polls = pd.read_csv('data/house_polls_merged.csv')
elections = pd.read_csv('data/1976-2022-house.csv')

In [3]:
elections_2018_2022 = pd.read_csv('data/2018-2022-house.csv')
elections_2018_2022.index = [i for i in range(len(elections_2018_2022.values))]
groupby = elections_2018_2022.groupby(['year', 'state', 'district'])

In [4]:
print(set(elections_2018_2022['stage']))

{'GEN'}


In [5]:

election_df = pd.DataFrame(columns=['year', 'state', 'district', 'c1', 'c2', 'c1_party', 'c2_party', 'c1_votes', 'c2_votes', 'winner', 'winner_party'])

# print(election_df.shape)
groups = list(groupby)




# election key: {year, state, district}
# group is tuple of (election key, DF with canidate instances)
# print(groups[0][1].columns)
for group in groups:
    df = group[1]
    c1_idx = df['totalvotes'].idxmax()
    c1 = df['candidate'][c1_idx]
    c1_party = df['party'][c1_idx]
    c1_votes = df['totalvotes'][c1_idx]

    df = df.drop(index=c1_idx)
    try:
        c2_idx = df['totalvotes'].idxmax()
        c2 = df['candidate'][c2_idx]
        c2_party = df['party'][c2_idx]
        c2_votes = df['totalvotes'][c2_idx]
    except:
        c2, c2_party, c2_votes = None, None, None
    
   
    new_row = [group[0][0], group[0][1], group[0][2], c1, c2, c1_party, c2_party, c1_votes, c2_votes, c1, c1_party]
    election_df.loc[len(election_df)] = new_row
    

# print(election_df.columns)
# print(election_df.shape)
# print(election_df.values[:10])



In [6]:

election_df.to_csv('data/elections-2018-2022.csv', index=False)

In [14]:
polls = pd.read_csv('data/house_polls_merged.csv')
polls['candidate_name'] = polls['candidate_name'].apply(lambda x: x.upper())
polls['state'] = polls['state'].apply(lambda x: x.upper())

print(polls.columns)
groupby = polls.groupby(['state', 'seat_number', 'cycle'])

groups = list(groupby)


# atts that good be good (beyond obvious):
# poll_id, pollster_id, sponsor_id, pollster_rating_id, pollscore, methodology, transparency score, sample_size, partisan
# sample_size

poll_df = pd.DataFrame(columns=['year', 'state', 'district', 'c1', 'c2', 'c1_party', 'c2_party', 'c1_pct', 'c2_pct'])


print(groups[0][0])
for group in groups:
    df = group[1]
    c1_idx = df['pct'].idxmax()
    c1 = df['candidate_name'][c1_idx]
    c1_party = df['party'][c1_idx]
    c1_pct = df['pct'][c1_idx]

    df = df.drop(index=c1_idx)
    try:
        c2_idx = df['pct'].idxmax()
        c2 = df['candidate_name'][c2_idx]
        c2_party = df['party'][c2_idx]
        c2_pct = df['pct'][c2_idx]
    except:
        c2, c2_party, c2_votes = None, None, None
    
   
    new_row = [group[0][2], group[0][0], group[0][1], c1, c2, c1_party, c2_party, c1_pct, c2_pct]
    poll_df.loc[len(poll_df)] = new_row





Index(['poll_id', 'pollster_id', 'pollster', 'sponsor_ids', 'sponsors',
       'display_name', 'pollster_rating_id', 'pollster_rating_name',
       'numeric_grade', 'pollscore', 'methodology', 'transparency_score',
       'state', 'start_date', 'end_date', 'sponsor_candidate_id',
       'sponsor_candidate', 'sponsor_candidate_party', 'endorsed_candidate_id',
       'endorsed_candidate_name', 'endorsed_candidate_party', 'question_id',
       'sample_size', 'population', 'subpopulation', 'population_full',
       'tracking', 'created_at', 'notes', 'url', 'url_article', 'url_topline',
       'url_crosstab', 'source', 'internal', 'partisan', 'race_id', 'cycle',
       'office_type', 'seat_number', 'seat_name', 'election_date', 'stage',
       'nationwide_batch', 'ranked_choice_reallocated', 'ranked_choice_round',
       'hypothetical', 'party', 'answer', 'candidate_id', 'candidate_name',
       'pct'],
      dtype='object')
('ALABAMA', 2, 2024)


In [ ]:
print(poll_df.columns)
print(election_df.columns)

print(len(poll_df.values))



Index(['year', 'state', 'district', 'c1', 'c2', 'c1_party', 'c2_party',
       'c1_pct', 'c2_pct'],
      dtype='object')
Index(['year', 'state', 'district', 'c1', 'c2', 'c1_party', 'c2_party',
       'c1_votes', 'c2_votes', 'winner', 'winner_party'],
      dtype='object')
484


In [ ]:
# joined_df = election_df.join(poll_df, on=['year', 'state', 'district'], how='inner')

joined_df = pd.merge(election_df, poll_df, on=['year', 'state', 'district'])

print(len(joined_df.values))

joined_df.to_csv('joined_table.csv')


print(joined_df['winner_party'].value_counts())

plt.bar(joined_df['winner_party'].value_counts(), height=0)

363
REPUBLICAN                        192
DEMOCRAT                          120
LIBERTARIAN                        21
CONSERVATIVE                        7
DEMOCRATIC-FARMER-LABOR             5
NO PARTY AFFILIATION                2
INDEPENDENT AMERICAN                2
GREEN                               2
UNITY PARTY OF COLORADO             1
INDEPENDENT                         1
WRITE-IN (INDEPENDENT)              1
LEGAL MARIJUANA NOW                 1
GRASSROOTS - LEGALIZE CANNABIS      1
CANNOT BE BOUGHT                    1
WOMEN'S EQUALITY                    1
REFORM                              1
NO  AFFILIATION                     1
UNAFFILIATED                        1
CONSTITUTION                        1
Name: winner_party, dtype: int64


TypeError: bar() missing 1 required positional argument: 'height'